In [28]:
import numpy as np
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import warnings
import pickle

import optuna
from sklearn.model_selection import cross_val_score
optuna.logging.set_verbosity(optuna.logging.WARNING)
warnings.filterwarnings("ignore")

In [29]:
# The directory for top level folder
dir_ = "/home/sugam/Work/20-29 Deep Learning/22 Projects/Optimization of Energy Using AIML"

processed_data_dir = dir_+'/data/Processed/'
model_dir = dir_+'/models/'

In [30]:
# Load your dataset
df = pd.read_pickle(processed_data_dir + "train_df_energy_final.pkl")

In [31]:
df.head()

,load_x,year,month,day,hour,minute,weekday,week_num,is_weekdend,load_min,...,rolling_mean_tmp_1_30,rolling_mean_tmp_1_60,rolling_mean_tmp_7_7,rolling_mean_tmp_7_14,rolling_mean_tmp_7_30,rolling_mean_tmp_7_60,rolling_mean_tmp_14_7,rolling_mean_tmp_14_14,rolling_mean_tmp_14_30,rolling_mean_tmp_14_60
0,65.8125,2018,1,1,1,0,0,1,0,38.53125,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,65.6875,2018,1,1,1,15,0,1,0,38.53125,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,66.0000,2018,1,1,1,30,0,1,0,38.53125,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,65.1875,2018,1,1,1,45,0,1,0,38.53125,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,70.6875,2018,1,1,2,0,0,1,0,38.53125,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
df.rename({"load_x":"load"},inplace=True)

In [33]:
# Check for remaining NaN values
nan_count = df.isna().sum().sum()
print(f"Total NaN values: {nan_count}")

Total NaN values: 0


In [34]:
df.columns

Index(['load_x', 'year', 'month', 'day', 'hour', 'minute', 'weekday',
       'week_num', 'is_weekdend', 'load_min', 'load_max', 'load_mean',
       'load_std', 'load_norm', 'load_lag_1', 'load_lag_2', 'load_lag_3',
       'load_lag_4', 'load_lag_5', 'load_lag_6', 'load_lag_7', 'load_lag_8',
       'load_lag_9', 'load_lag_10', 'load_lag_11', 'load_lag_12',
       'load_lag_13', 'load_lag_14', 'load_lag_15', 'rolling_mean_7',
       'rolling_std_7', 'rolling_mean_14', 'rolling_std_14', 'rolling_mean_30',
       'rolling_std_30', 'rolling_mean_60', 'rolling_std_60',
       'rolling_mean_180', 'rolling_std_180', 'rolling_mean_tmp_1_7',
       'rolling_mean_tmp_1_14', 'rolling_mean_tmp_1_30',
       'rolling_mean_tmp_1_60', 'rolling_mean_tmp_7_7',
       'rolling_mean_tmp_7_14', 'rolling_mean_tmp_7_30',
       'rolling_mean_tmp_7_60', 'rolling_mean_tmp_14_7',
       'rolling_mean_tmp_14_14', 'rolling_mean_tmp_14_30',
       'rolling_mean_tmp_14_60'],
      dtype='object')

In [12]:
# Split the dataset using fixed split 80% into training and 20% into testing
train_size = int(len(df) * 0.8)
train_data, test_data = df[:train_size], df[train_size:]
target_columns = ["load_x"] 

X_train, y_train = train_data.drop(target_columns, axis=1), train_data[target_columns]
X_test, y_test = test_data.drop(target_columns, axis=1), test_data[target_columns]

KeyError: "['load'] not found in axis"

In [29]:
print(f"Shape of Train data: {X_train.shape}")
print(f"Shape of Test data: {X_test.shape}")

Shape of Train data: (82438, 13)
Shape of Test data: (20610, 13)


In [30]:
xgb_model = XGBRegressor()
xgb_model.fit(X_train, y_train)
xgb_predictions = xgb_model.predict(X_valid)
xgb_rmse = mean_squared_error(y_valid, xgb_predictions, squared=False)
print(f'XGBoost RMSE: {xgb_rmse}')

XGBoost RMSE: 9.3037748336792


In [ ]:
def objective(trial):
    criterion = trial.suggest_categorical("criterion",["gini","entropy"])
    max_depth = trial.suggest_int("max_depth",2,32,log=True)
    n_estimators = trial.suggest_int("n_estimators",100,500)
    boosters = trial.suggest_categorical("boosters",["gbtree","gblinear","dart"])
    eta = trial.suggest_float("eta",0.1,1)
    learning_rate = trial.suggest_float("learning_rate",0.001,1,log=True)
    min_child_weight = trial.suggest_int("min_child_weight",1,10)
    gamma = trial.suggest_float("gamma",0.0,0.5)
    
    xgb_model = XGBRegressor(criterion = criterion,
                            max_depth = max_depth,
                            n_estimator = n_estimators,
                            verbosity=1,
                            boosters = boosters,
                            eta = eta,
                            learning_rate = learning_rate,
                            min_child_weight = min_child_weight,
                            gamma = gamma)
    
    xgb_model.fit(X_train,y_train)
    rmse_score = mean_squared_error(y_train,xgb_model.predict(X_train),squared=False)
    return rmse_score


study = optuna.create_study(direction="minimize",
                            study_name="xgb_study")
study.optimize(objective, n_trials=20)
trial = study.best_trial
print(f"RMSE: {trial.value}")
print(f"Params: {trial.params}")

In [17]:
xgb_model = XGBRegressor(criterion='entropy',
                         max_depth =  18,
                         n_estimators = 173,
                         boosters = 'gblinear',
                         eta = 0.8321535613093416, 
                         learning_rate = 0.9327551352534427,
                         min_child_weight = 5,
                         gamma = 0.0014464638207411923)
xgb_model.fit(X_train, y_train)
xgb_predictions = xgb_model.predict(X_test)
xgb_rmse = mean_squared_error(y_test, xgb_predictions, squared=False)
print(f'XGBoost RMSE: {xgb_rmse}')

XGBoost RMSE: 3.1189017295837402


In [37]:
model_name = model_dir+'xgb_model_train_df_part_1.bin'
pickle.dump(xgb_model, open(model_name, 'wb'))